<a href="https://colab.research.google.com/github/wooridle/mlp_lecture/blob/main/6_multi_class_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [4]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


# 로우레벨 구현

y_one_hot = torch.zeros(8, 3)

y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

print(y_one_hot.shape)

torch.Size([8, 3])


#### 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)

b = torch.zeros(1, requires_grad=True)

#### optimizer 설정

optimizer = optim.SGD([W, b], lr=0.1)

#### 학습

nb_epochs = 1000

for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

# F.cross_entropy 사용

#### 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)

b = torch.zeros(1, requires_grad=True)

#### optimizer 설정

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

# torch 클래스 형태로 구현

class SoftmaxClassifierModel(nn.Module):

    def __init__(self):

        super().__init__()
        
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

model = SoftmaxClassifierModel()

#### optimizer 설정

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))